In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)

In [2]:
mart = pd.read_excel('../new_data/상업시설/대규모점포정보.xlsx')
mart = mart.dropna(subset=['소재지전체주소'])
mart = mart.loc[mart['소재지전체주소'].str.contains('평택시')][['사업장명','소재지전체주소']].dropna().reset_index().drop(columns=['index'])

/Users/hdk/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### 위도 경도 생성 (NAVER API)

In [3]:
import sys
sys.path.append("..")  # 상위 디렉토리를 sys.path에 추가

import requests
import pandas as pd
from mySetting import Client_ID, Client_Secret

# 네이버 API 키
client_id = Client_ID  # 클라이언트 ID
client_secret = Client_Secret  # 클라이언트 Secret

def get_coordinates(address):
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'query': address,
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data['addresses']:
        x = data['addresses'][0]['x']
        y = data['addresses'][0]['y']
        return (x, y)
    else:
        return (None, None)

In [4]:
mart['위도'] = pd.NA
mart['경도'] = pd.NA

for i, address in enumerate(mart['소재지전체주소']):
    lng, lat = get_coordinates(address)
    mart.at[i, '위도'] = lat
    mart.at[i, '경도'] = lng

In [5]:
mart.rename(columns={'소재지전체주소':'주소','사업장명':'이름'}, inplace=True)

In [6]:
mart.to_csv('../preprocessed/상업시설/마트.csv', index=False)